In [1]:
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import re

# Reading in my poker tournament data
df_poker = pd.read_csv("Resources/Jamie_Poker.csv")
# Setting focus user
user = 'JJin91'

# Hand history df
df_hh = pd.DataFrame(columns=['Tournament_ID','Hand_Count','Players_At_Table','User_Position','Chip_Count',
                              'User_Cards','PreFlop','Flop_Cards','Flop','Turn_Card','Turn','River_Card','River',
                              'Showdown','Total_Pot','Summary'])
df_poker

,Tournament_ID,Date,Buy_In,Prize_Pool,Placement,Entrants,Award
0,1597683884,2017/05/02 19:00:00 ET,20,1983.80,7,109,79.35
1,1597684008,2017/05/05 13:30:00 ET,15,1431.00,8,106,46.50
2,1597684021,2017/05/05 19:00:00 ET,20,2093.00,1,115,481.46
3,1597684053,2017/05/06 13:30:00 ET,15,1444.50,15,107,18.77
4,1597684117,2017/05/07 21:00:00 ET,5,500.00,5,106,32.50
5,1622537113,2017/05/09 18:00:00 ET,10,1046.50,17,115,21.45
6,1622537329,2017/05/14 13:30:00 ET,15,1606.50,10,119,34.53
7,1622573245,2017/05/24 21:00:00 ET,5,541.45,14,119,11.64
8,1622573280,2017/05/25 18:00:00 ET,10,1528.80,13,168,22.93
9,1648956582,2017/06/19 19:00:00 ET,20,2875.60,13,158,43.13


In [2]:
# Position finder function based on button position, user position, and number of seats
def PlayerPositionFinder(button_seat_position,user_seat_position,num_of_players):
    if num_of_players == 2:
        if button_seat == user_seat:
            return 'SB'
        else:
            return 'BB'
    else:
        if button_seat == user_seat:
            return 'BT'
        elif button_seat_position - user_seat_position == -1 or button_seat_position - user_seat_position == num_of_players-1:
            return 'SB'
        elif button_seat_position - user_seat_position == -2 or button_seat_position - user_seat_position == num_of_players-2:
            return 'BB'
        elif button_seat_position - user_seat_position == -3 or button_seat_position - user_seat_position == num_of_players-3:
            return 'UTG'
        elif button_seat_position - user_seat_position == 1 or button_seat_position - user_seat_position == -(num_of_players-1):
            return 'CO'
        elif button_seat_position - user_seat_position == -4 or button_seat_position - user_seat_position == num_of_players-4:
            return 'UTG+1'
        elif button_seat_position - user_seat_position == 2 or button_seat_position - user_seat_position == -(num_of_players-2):
            return 'HJ'
        elif button_seat_position - user_seat_position == -5 or button_seat_position - user_seat_position == num_of_players-5:
            return 'MP'
        elif button_seat_position - user_seat_position == -6 or button_seat_position - user_seat_position == num_of_players-6:
            return 'MP+1'
        else:
            return 'table position error'

In [3]:
class HandActionParser(object):
    def __init__(self, action):
        self.action = action
        
    #def FlopCards(self):
        
    def PreFlop(self):
        if "*** FLOP ***" in self.action:
            return "-PREFLOP- " + self.action.split("*** FLOP ***")[0]
        else:
            return "-PREFLOP- " + self.action
        
    def Flop(self):
        if not "*** FLOP ***" in self.action:
            return 'NA'
        elif "*** TURN ***" in self.action:
            return "-FLOP- " + self.action.split("*** FLOP ***")[1].split("*** TURN ***")[0]
        else:
            return "-FLOP- " + self.action.split("*** FLOP ***")[1]
    
    def Turn(self):
        if not "*** TURN ***" in self.action:
            return 'NA'
        elif "*** RIVER ***" in self.action:
            return "-TURN- " + self.action.split("*** TURN ***")[1].split("*** RIVER ***")[0]
        else:
            return "-TURN- " + self.action.split("*** TURN ***")[1]
    
    def River(self):
        if "*** RIVER ***" in self.action:
            if "*** SHOW DOWN ***" in self.action:
                return "-RIVER- " + self.action.split("*** RIVER ***")[1].split('*** SHOW DOWN ***')[0]
            else:
                return "-RIVER- " + self.action.split("*** RIVER ***")[1]
        else:
            return 'NA'
        
    def Showdown(self):
        if "*** SHOW DOWN ***" in self.action:
            return self.action.split('*** SHOW DOWN ***')[1]
        else:
            return 'NA'

In [4]:
def TableCards(stage):
    if "-FLOP-" in stage:
        return re.search("\w\w\s\w\w\s\w\w", stage).group().split(' ')
    elif "-TURN-" in stage:
        return re.search("\[\w\w\]", stage).group()[1:3]
    elif "-RIVER-" in stage:
        return re.search("\[\w\w\]", stage).group()[1:3]
    else:
        return 'NA'

In [5]:
def ActionsList(stage):
    actions_list = [step for step in stage.split('\n')]
    if len(actions_list) == 2 or stage == 'NA':
        return 'NA'
    elif "-PREFLOP-" in stage:
        return actions_list[2:len(actions_list)-1]
    else:
        return actions_list[1:len(actions_list)-1]

In [6]:
tourny = []
# Reading all hand history text files
for i in range(len(df_poker)): 
    tourny_temp = open("Resources/{}.txt".format(df_poker['Tournament_ID'].iloc[i]),'r').read()
    # Removing bottom contact message
    tourny_temp = tourny_temp.split("\n\n\n\n", 1)[0]
    # Splitting by hands and removing top hand history request message
    tourny_temp_split = re.split('\*{11}\s\#\s\d*\s\*{14}',tourny_temp)[1:]
    tourny_temp_split
    tourny.append(tourny_temp_split)

index = 0
for i in range(len(tourny)):
    hand_count = 1
    for ii in range(len(tourny[i])):
        # Tourny ID to df_hh
        tourny_id = re.search("Tournament #\d{10}", tourny[i][ii]).group().split(' ')[1][1:]
        df_hh.set_value(index, 'Tournament_ID', tourny_id)
        
        seat_list = [int(i.split(" ")[1]) for i in re.findall("Seat\s\d",tourny[i][ii].split("*** SUMMARY ***")[1])]   
        button_seat = int(re.search("Seat \#\d is the button", tourny[i][ii]).group().split(" #")[1].split(" ")[0])
        user_seat = int(re.search("Seat \d\: " + user, tourny[i][ii]).group().split(": ")[0].split(" ")[1])
        
        # Players at table of tourny[i] hand[ii] to df_hh
        df_hh.set_value(index, 'Players_At_Table', len(seat_list))
        
        # Find user position using above position finder funciton
        try:
            user_position = PlayerPositionFinder(seat_list.index(button_seat),seat_list.index(user_seat),len(seat_list))
            df_hh.set_value(index, 'User_Position', user_position)
        except:
            print("tournament {} hand {} position fail".format(df_poker['Tournament_ID'].iloc[i],hand_count))

        # Hand number of tourny[i] hand[ii] to df_hh
        df_hh.set_value(index, 'Hand_Count', hand_count)
        
        # Chip count before start of hand to df_hh
        df_hh.set_value(index, 'Chip_Count', int(re.search('(?<=Seat \d\: {}\s\()\d*'.format(user),tourny[i][ii]).group()))
        
        # Action only - no summary
        current_action = HandActionParser(tourny[i][ii].split("*** HOLE CARDS ***")[1].split('*** SUMMARY ***')[0])
        preflop = current_action.PreFlop()
        flop = current_action.Flop()
        turn = current_action.Turn()
        river = current_action.River()
        showdown = current_action.Showdown()
        
        # User cards to df_hh
        try:
            df_hh.set_value(index, 'User_Cards', re.search('(?<={}\s\[)\w\w\s\w\w'.format(user),preflop).group().split(' '))
        except:
            print("tournament {} hand {} User cards fail".format(df_poker['Tournament_ID'].iloc[i],hand_count))
        
        # PreFlop, Flop, Turn, & River Action to df_hh
        try:
            df_hh.set_value(index, 'PreFlop', ActionsList(preflop))
            df_hh.set_value(index, 'Flop', ActionsList(flop))
            df_hh.set_value(index, 'Turn', ActionsList(turn))
            df_hh.set_value(index, 'River', ActionsList(river))
            df_hh.set_value(index, 'Showdown', ActionsList(showdown))
        except:
            print("tournament {} hand {} Table cards fail".format(df_poker['Tournament_ID'].iloc[i],hand_count))
            
        # Flop, Turn, & River Table Cards to df_hh
        try:
            df_hh.set_value(index, 'Flop_Cards', TableCards(flop))
            df_hh.set_value(index, 'Turn_Card', TableCards(turn))
            df_hh.set_value(index, 'River_Card', TableCards(river))
        except:
            print("tournament {} hand {} Table cards fail".format(df_poker['Tournament_ID'].iloc[i],hand_count))
        
        # Total Pot to df_hh
        try:
            pot = re.search("(?<=Total\spot\s)\d*", tourny[i][ii].split('*** SUMMARY ***')[1]).group()
            df_hh.set_value(index, 'Total_Pot', pot)
        except:
            print("tournament {} hand {} Summary fail".format(df_poker['Tournament_ID'].iloc[i],hand_count))
        
        # Summary to df_hh
        try:
            summary = [i for i in tourny[0][3].split('*** SUMMARY ***')[1].split('\n') if 'Seat' in i]
            df_hh.set_value(index, 'Summary', summary)
        except:
            print("tournament {} hand {} Summary fail".format(df_poker['Tournament_ID'].iloc[i],hand_count))
        
        index += 1
        hand_count += 1
        
df_hh.head()

tournament 1622537113 hand 94 position fail
tournament 1622537113 hand 94 User cards fail
tournament 1648956862 hand 2 position fail
tournament 1648956862 hand 2 User cards fail
tournament 1701793695 hand 314 position fail
tournament 1701793695 hand 314 User cards fail


,Tournament_ID,Hand_Count,Players_At_Table,User_Position,Chip_Count,User_Cards,PreFlop,Flop_Cards,Flop,Turn_Card,Turn,River_Card,River,Showdown,Total_Pot,Summary
0,1597683884,1,8,HJ,10000,"[4d, 5c]","[moosedeer17: folds=20, Whiteivey64: folds=20,...",NA,NA,NA,NA,NA,NA,NA,1070,[Seat 1: j2157 showed [Jd Qc] and lost with tw...
1,1597683884,2,8,MP,9960,"[3s, 8h]","[Whiteivey64: folds=20, iFoldN0T: folds=20, JJ...",NA,NA,NA,NA,NA,NA,NA,1400,[Seat 1: j2157 showed [Jd Qc] and lost with tw...
2,1597683884,3,8,UTG+1,9910,"[Jd, 6h]","[iFoldN0T: folds=20, JJin91: folds=20, DaleTDo...",NA,NA,NA,NA,NA,NA,NA,1200,[Seat 1: j2157 showed [Jd Qc] and lost with tw...
3,1597683884,4,8,UTG,9860,"[5s, Qs]","[JJin91: folds=20, DaleTDobak: calls 400, j215...","[4d, Qh, 9d]","[iFoldN0T: checks=20, j2157: checks=20, moosed...",Kc,NA,9h,NA,"[j2157: shows [Jd Qc] (two pair, Queens and Ni...",18482,[Seat 1: j2157 showed [Jd Qc] and lost with tw...
4,1597683884,5,7,BB,9810,"[9d, Js]","[DaleTDobak: raises 1200 to 1600, ruskie85: fo...",NA,NA,NA,NA,NA,NA,NA,1350,[Seat 1: j2157 showed [Jd Qc] and lost with tw...


In [7]:
# action1 = tourny[0][3].split("*** HOLE CARDS ***")[1].split('*** SUMMARY ***')[0]
# hand_action = HandActionParser(action1)
# preflop = hand_action.PreFlop()
# flop = hand_action.Flop()
# turn = hand_action.Turn()
# river = hand_action.River()

# preflop

In [8]:
'''
Wanted JSON Format:
grouped by Tournament ID
PreFlop, Flop, Turn, River, Showdown, and Summary columns must be in dictionary form
Tournament_ID,Hand_Count,Players_At_Table, and Total_Pot must be integers
values with NA must not be present in JSON
'''

'\nWanted JSON Format:\ngrouped by Tournament ID\nPreFlop, Flop, Turn, River, Showdown, and Summary columns must be in dictionary form\nTournament_ID,Hand_Count,Players_At_Table, and Total_Pot must be integers\nvalues with NA must not be present in JSON\n'